In [18]:
#Import required libraries
import pandas as pd
import altair as alt
alt.renderers.enable('notebook')
alt.data_transformers.enable('default', max_rows=None)
import warnings
warnings.filterwarnings('ignore')
from sklearn import decomposition
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import numpy as np

In [9]:
# Full Corpus
full_corpus_df = pd.read_csv('../data/arab_observer_corpus_cleaned.csv')
#Distinguish origin of data 
full_corpus_df.type[full_corpus_df.type == 'hathi_trust'] = 'HathiTrust Volumes'
full_corpus_df.type[full_corpus_df.type == 'collected_issues'] = 'Collected Issues'
full_corpus_df.sort_values(by=['datetime', 'binned', 'page_number'], inplace=True)
#Order datasets temporally
full_corpus_df['date_bin'] = full_corpus_df.date.astype(str)+'_'+full_corpus_df.binned.astype(str)
full_corpus_df['date_page'] = full_corpus_df.date.astype(str)+'_'+full_corpus_df.page_number.astype(str)
full_corpus_df['datetime'] = pd.to_datetime(full_corpus_df['date'], format='%Y-%B-%d', errors='coerce')
full_corpus_dates = full_corpus_df.date.unique().tolist()

In [33]:
ao_1960 = full_corpus_df.query('date.str.contains("1961")')

In [34]:
ao_1960.dropna(subset=['cleaned_spacy_text'], inplace=True)

In [35]:
tfidf_model = TfidfVectorizer(ngram_range=(1,2), lowercase=False)
features = tfidf_model.fit_transform(ao_1960.cleaned_spacy_text.tolist())

features_dtm = features.toarray()

In [19]:
dist = euclidean_distances(features_dtm)
cos = 1- cosine_similarity(features_dtm)
norms = np.sqrt(np.sum(features_dtm * features_dtm, axis=1, keepdims=True))
dtm_normed = features_dtm / norms
sim = np.dot(dtm_normed, dtm_normed.T)
sim[1, 3]

0.0

In [36]:

num_topics = 10
num_top_words = 20
clf = decomposition.NMF(n_components=num_topics, random_state=1)
doctopic = clf.fit_transform(features_dtm)

KeyboardInterrupt: 

In [ ]:
vocab = np.array(tfidf_model.get_feature_names())

In [ ]:
topic_words = []
for topic in clf.components_:
    word_idx = np.argsort(topic)[::-1][0:num_top_words]
    topic_words.append([vocab[i] for i in word_idx])
topic_words

In [32]:
# model = decomposition.NMF(init=self.init_strategy, n_components=k, max_iter=self.max_iters, random_state = self.random_seed)
# 		self.W = model.fit_transform(X)
# 		self.H = model.components_	
        
# top_indices = np.argsort( self.H[topic_index,:] )[::-1]
# 		# truncate if necessary
# 		if top < 1 or top > len(top_indices):
# 			return top_indices
# 		return top_indices[0:top]

# 	def generate_partition( self ):
# 		if self.W is None:
# 			raise ValueError("No results for previous run available")
# 		return np.argmax( self.W, axis = 1 ).flatten().tolist()	
# def generate_doc_rankings( W ):
# 	'''
# 	Rank document indices, based on values in a W factor matrix produced by NMF.
# 	'''
# 	doc_rankings = []
# 	k = W.shape[1]
# 	for topic_index in range(k):
# 		w = np.array( W[:,topic_index] )
# 		top_indices = np.argsort(w)[::-1]
# 		doc_rankings.append(top_indices)
# 	return doc_rankings